In [3]:
import pandas as pd
import numpy as np

col_headers = ['TrackID', 'UserID', 'ItemID', 'AlbumID', 'ArtistID', 'Genre1ID', 'Genre2ID', 'Genre3ID', 'AlbumRating', 'ArtistRating', 'Genre1Rating', 'Genre2Rating', 'Genre3Rating']
clean_data = pd.read_csv('./training_gross_statistics_matrix_fact_1207.csv')
clean_data = clean_data.drop(['Genre4ID', 'Genre5ID', 'orig_genre4_rating', 'orig_genre5_rating'], axis=1)
clean_data.columns = col_headers

clean_data = clean_data.fillna(value=0)
clean_data['AlbumID'].fillna(value=0, inplace=True)
clean_data['UserID'] = clean_data['UserID'].astype(np.float64)
clean_data['ItemID'] = clean_data['ItemID'].astype(np.float64)
clean_data['AlbumID'] = clean_data['AlbumID'].astype(object)
clean_data['ArtistID'] = clean_data['ArtistID'].astype(object)
clean_data['Genre1ID'] = clean_data['Genre1ID'].astype(np.float64)
clean_data['Genre2ID'] = clean_data['Genre2ID'].astype(np.float64)
clean_data['Genre3ID'] = clean_data['Genre3ID'].astype(np.float64)
clean_data['AlbumRating'] = clean_data['AlbumRating'].astype(np.float64)
clean_data['ArtistRating'] = clean_data['ArtistRating'].astype(np.float64)
clean_data['Genre1Rating'] = clean_data['Genre1Rating'].astype(np.float64)
clean_data['Genre2Rating'] = clean_data['Genre2Rating'].astype(np.float64)
clean_data['Genre3Rating'] = clean_data['Genre3Rating'].astype(np.float64)

clean_data['AlbumID'] = clean_data['AlbumID'].replace('None', np.NaN)
clean_data['ArtistID'] = clean_data['ArtistID'].replace('None', np.NaN)
clean_data = clean_data.fillna(0)
clean_data['AlbumID'] = clean_data['AlbumID'].astype(np.float64)
clean_data['ArtistID'] = clean_data['ArtistID'].astype(np.float64)

print(clean_data.head())


         TrackID    UserID    ItemID   AlbumID  ArtistID  Genre1ID  Genre2ID  \
0  199810_208019  199810.0  208019.0  209288.0       0.0  131552.0  173467.0   
1   199810_74139  199810.0   74139.0  277282.0  271146.0  131552.0  173467.0   
2    199810_9903  199810.0    9903.0       0.0       0.0  131552.0  173467.0   
3  199810_242681  199810.0  242681.0  190640.0  244574.0  131552.0  173467.0   
4   199810_18515  199810.0   18515.0  146344.0   33168.0  131552.0  173467.0   

   Genre3ID  AlbumRating  ArtistRating  Genre1Rating  Genre2Rating  \
0   48505.0          0.0           0.0           0.0           0.0   
1   48505.0          0.0           0.0           0.0          80.0   
2   48505.0          0.0           0.0           0.0           0.0   
3   48505.0          0.0           0.0           0.0           0.0   
4   48505.0          0.0          70.0           0.0           0.0   

   Genre3Rating  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4       

In [18]:
import numpy as np
from sklearn import metrics
import math

def update_p_q(nP, nQ, alpha, beta, K, R):
    shape = R.shape
    nRow = shape[0]
    nCol = shape[1]
    Q_T = nQ.T
    
    error = 0
    mse = 0
    err_total = 0
    for i in range(0,nRow):
        for j in range(0,nCol):
            if R[i][j] > 0:
                error = R[i][j] - np.dot(nP[i,:],Q_T[:,j])
                mse = error**2
                
                for k in range(0,K):
                    term_2_q = 2*error*nP[i][k]
                    term_3_q = beta*Q_T[k][j]
                    term_2_p = 2*error*Q_T[k][j]
                    term_3_p = beta*nP[i][k]
                    
                    nP[i][k] = nP[i][k] + (alpha*(term_2_p-term_3_p))
                    Q_T[k][j] = Q_T[k][j] + (alpha*(term_2_q-term_3_q))
                    
                    p_q = np.dot(nP[i,:],Q_T[:,j])
                    err_total = err_total + mse + ((beta/2) * (pow(nP[i][k],2) + (pow(Q_T[k][j],2))))
            
    nQ = Q_T.T
    return (nP, nQ, err_total)

def matrix_factorization(R,P,Q,K, steps, alpha, beta):
    epsilon = 9600000 #1,569,245 #1e-6 9597502
    error_tot = epsilon
    nP = P
    nQ = Q

    for step in range(0, steps):
        (nP, nQ, error_tot) = update_p_q(nP, nQ, alpha, beta, K, R)
        print("loss: ", error_tot)
        
        # break the loop
        if (error_tot < epsilon):
            break

    return (nP, nQ)

In [19]:
rows = clean_data.drop(['TrackID', 'UserID', 'ItemID', 'AlbumID', 'ArtistID', 'Genre1ID', 'Genre2ID', 'Genre3ID'], axis=1)
print(rows.head())
rows = rows.to_numpy()
print(rows)
R = rows
shape = R.shape
nRow = shape[0]
nCol = shape[1]

K = 2
P = np.random.rand(nRow,K)
Q = np.random.rand(nCol,K)

steps = 10000
alpha = 0.0002
beta = 0.02
nP, nQ = matrix_factorization(R,P,Q,K, steps, alpha, beta)
Q_T = nQ.T
p_q = np.dot(nP,Q_T)
print(p_q[:5])

mf_data = pd.DataFrame(p_q)

   AlbumRating  ArtistRating  Genre1Rating  Genre2Rating  Genre3Rating
0          0.0           0.0           0.0           0.0           0.0
1          0.0           0.0           0.0          80.0           0.0
2          0.0           0.0           0.0           0.0           0.0
3          0.0           0.0           0.0           0.0           0.0
4          0.0          70.0           0.0           0.0           0.0
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0. 80.  0.]
 [ 0.  0.  0.  0.  0.]
 ...
 [90. 90.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [90. 90.  0.  0.  0.]]
loss:  515846850.5558741
loss:  114901650.01956443
loss:  71017646.69919285
loss:  66004029.950456426
loss:  62420322.11036521
loss:  59035126.95721389
loss:  54983516.63416605
loss:  49464608.285389915
loss:  42791456.17441432
loss:  36522870.68650106
loss:  31802273.284680888
loss:  28702551.5348096
loss:  26822068.64473444
loss:  25640836.42781425
loss:  24787246.804550108
loss:  24092009.208950177
loss:  23492658.19941902


In [25]:
col_header = ['AlbumRating_MF', 'ArtistRating_MF', 'Genre1Rating_MF', 'Genre2Rating_MF', 'Genre3Rating_MF', 'TrackID', 'UserID', 'ItemID', 'AlbumID', 'ArtistID', 'Genre1ID', 'Genre2ID', 'Genre3ID', 'AlbumRating', 'ArtistRating', 'Genre1Rating', 'Genre2Rating', 'Genre3Rating']
mf_data.columns = col_header
mf_data.to_csv('./1211_mf_raw.csv', index=False, header=col_headers)
print(clean_data.head())
print(mf_data.head())

mf_data['TrackID'] = clean_data['TrackID']
mf_data['UserID'] = clean_data['UserID']
mf_data['ItemID'] = clean_data['ItemID']
mf_data['AlbumID'] = clean_data['AlbumID']
mf_data['ArtistID'] = clean_data['ArtistID']
mf_data['Genre1ID'] = clean_data['Genre1ID']
mf_data['Genre2ID'] = clean_data['Genre2ID']
mf_data['Genre3ID'] = clean_data['Genre3ID']
mf_data['AlbumRating'] = clean_data['AlbumRating']
mf_data['ArtistRating'] = clean_data['ArtistRating']
mf_data['Genre1Rating'] = clean_data['Genre1Rating']
mf_data['Genre2Rating'] = clean_data['Genre2Rating']
mf_data['Genre3Rating'] = clean_data['Genre3Rating']

col_header = ['AlbumRating_MF', 'ArtistRating_MF', 'Genre1Rating_MF', 'Genre2Rating_MF', 'Genre3Rating_MF', 'TrackID', 'UserID', 'ItemID', 'AlbumID', 'ArtistID', 'Genre1ID', 'Genre2ID', 'Genre3ID', 'AlbumRating', 'ArtistRating', 'Genre1Rating', 'Genre2Rating', 'Genre3Rating']
mf_data.to_csv('./1211_mf.csv', index=False, header=col_headers)

scores_df = mf_data.drop(['TrackID', 'UserID', 'ItemID', 'AlbumID', 'ArtistID', 'Genre1ID', 'Genre2ID', 'Genre3ID', 'AlbumRating_MF', 'ArtistRating_MF', 'Genre1Rating_MF', 'Genre2Rating_MF', 'Genre3Rating_MF'], axis=1)

min_scores = scores_df.min(axis=1)
max_scores = scores_df.max(axis=1)
mean_scores = scores_df.mean(axis=1)
variance_scores = scores_df.var(axis=1)

mf_data['MinScore'] = min_scores
mf_data['MaxScore'] = max_scores
mf_data['MeanScore'] = mean_scores
mf_data['VarScore'] = variance_scores

scores_df_mf = mf_data.drop(['TrackID', 'UserID', 'ItemID', 'AlbumID', 'ArtistID', 'Genre1ID', 'Genre2ID', 'Genre3ID', 'AlbumRating', 'ArtistRating', 'Genre1Rating', 'Genre2Rating', 'Genre3Rating', 'MinScore', 'MaxScore', 'MeanScore', 'VarScore'], axis=1)

min_scores = scores_df_mf.min(axis=1)
max_scores = scores_df_mf.max(axis=1)
mean_scores = scores_df_mf.mean(axis=1)
variance_scores = scores_df_mf.var(axis=1)

mf_data['MinScore_MF'] = min_scores
mf_data['MaxScore_MF'] = max_scores
mf_data['MeanScore_MF'] = mean_scores
mf_data['VarScore_MF'] = variance_scores

#col_headers = ['TrackID', 'UserID', 'ItemID', 'AlbumID', 'ArtistID', 'Genre1ID', 'Genre2ID', 'Genre3ID', 'AlbumRating_MF', 'ArtistRating_MF', 'Genre1Rating_MF', 'Genre2Rating_MF', 'Genre3Rating_MF', 'AlbumRating', 'ArtistRating', 'Genre1Rating', 'Genre2Rating', 'Genre3Rating', 'MinScore', 'MaxScore', 'MeanScore', 'VarScore', 'MinScore_MF', 'MaxScore_MF', 'MeanScore_MF', 'VarScore_MF']
mf_data.to_csv('./1211_total.csv', index=False) #, header=col_headers)

print(mf_data.head())
print("reached here")

         TrackID    UserID    ItemID   AlbumID  ArtistID  Genre1ID  Genre2ID  \
0  199810_208019  199810.0  208019.0  209288.0       0.0  131552.0  173467.0   
1   199810_74139  199810.0   74139.0  277282.0  271146.0  131552.0  173467.0   
2    199810_9903  199810.0    9903.0       0.0       0.0  131552.0  173467.0   
3  199810_242681  199810.0  242681.0  190640.0  244574.0  131552.0  173467.0   
4   199810_18515  199810.0   18515.0  146344.0   33168.0  131552.0  173467.0   

   Genre3ID  AlbumRating  ArtistRating  Genre1Rating  Genre2Rating  \
0   48505.0          0.0           0.0           0.0           0.0   
1   48505.0          0.0           0.0           0.0          80.0   
2   48505.0          0.0           0.0           0.0           0.0   
3   48505.0          0.0           0.0           0.0           0.0   
4   48505.0          0.0          70.0           0.0           0.0   

   Genre3Rating  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4       

In [44]:
# Function to sort the list of tuples by its second item
def Sort_Tuple(tup):
    lst = len(tup)
    for i in range(0, lst):
        for j in range(0, lst-i-1):
            if (tup[j][1] > tup[j + 1][1]):
                temp = tup[j]
                tup[j]= tup[j + 1]
                tup[j + 1]= temp
    return tup

data = mf_data
print(data.head())

rows = []
sum_arr = []
user_arr = []
flag = 0
for ind in range(0, len(data)):  
    userID_trackID = data['TrackID'][ind]
    album_rating = data['AlbumRating_MF'][ind]
    artist_rating = data['ArtistRating_MF'][ind]
    genre1_rating = data['Genre1Rating_MF'][ind]
    genre2_rating = data['Genre2Rating_MF'][ind]
    genre3_rating = data['Genre3Rating_MF'][ind]
    sum_ratings = album_rating + artist_rating + genre1_rating + genre2_rating + genre3_rating
    sum_arr.append((userID_trackID, sum_ratings))
    user_arr.append(userID_trackID)
    
    if (len(sum_arr) == 6):
        sum_arr = Sort_Tuple(sum_arr)
        sorted_users = []
        for i in range(0, len(sum_arr)):
            sorted_users.append(sum_arr[i][0])
        users_0 = sorted_users[:3]
        
        for i in range(0, len(user_arr)):
            if (user_arr[i] in users_0):
                rating = 0
            else:
                rating = 1
            row = [user_arr[i], rating]
            rows.append(row)
        
        sum_arr = []
        user_arr = []

rows = np.array(rows) 
newData_cols = ['TrackID', 'Predictor']
newData = pd.DataFrame(rows)
newData.columns = newData_cols
print(newData.head())
newData.to_csv('./matrix_fact_predictions_1211.csv', index=False)

   AlbumRating_MF  ArtistRating_MF  Genre1Rating_MF  Genre2Rating_MF  \
0       11.690904        12.474678        13.945157        12.992632   
1       77.389346        80.694513        83.869803        79.879048   
2       14.254794        13.852148        10.914258        11.422376   
3        4.574390         4.590136         4.152230         4.137116   
4       72.024521        69.910350        54.788498        57.453849   

   Genre3Rating_MF        TrackID    UserID    ItemID   AlbumID  ArtistID  \
0        13.243756  199810_208019  199810.0  208019.0  209288.0       0.0   
1        80.586610   199810_74139  199810.0   74139.0  277282.0  271146.0   
2        11.039877    199810_9903  199810.0    9903.0       0.0       0.0   
3         4.087868  199810_242681  199810.0  242681.0  190640.0  244574.0   
4        55.480790   199810_18515  199810.0   18515.0  146344.0   33168.0   

   ...  Genre2Rating  Genre3Rating  MinScore  MaxScore  MeanScore  VarScore  \
0  ...           0.0     

In [45]:
# Function to sort the list of tuples by its second item
def Sort_Tuple(tup):
    lst = len(tup)
    for i in range(0, lst):
        for j in range(0, lst-i-1):
            if (tup[j][1] > tup[j + 1][1]):
                temp = tup[j]
                tup[j]= tup[j + 1]
                tup[j + 1]= temp
    return tup

data = mf_data
print(data.head())

rows = []
sum_arr = []
user_arr = []
flag = 0
for ind in range(0, len(data)):  
    userID_trackID = data['TrackID'][ind]
    album_rating = data['AlbumRating'][ind]
    artist_rating = data['ArtistRating'][ind]
    genre1_rating = data['Genre1Rating'][ind]
    genre2_rating = data['Genre2Rating'][ind]
    genre3_rating = data['Genre3Rating'][ind]
    sum_ratings = album_rating + artist_rating + genre1_rating + genre2_rating + genre3_rating
    sum_arr.append((userID_trackID, sum_ratings))
    user_arr.append(userID_trackID)
    
    if (len(sum_arr) == 6):
        sum_arr = Sort_Tuple(sum_arr)
        sorted_users = []
        for i in range(0, len(sum_arr)):
            sorted_users.append(sum_arr[i][0])
        users_0 = sorted_users[:3]
        
        for i in range(0, len(user_arr)):
            if (user_arr[i] in users_0):
                rating = 0
            else:
                rating = 1
            row = [user_arr[i], rating]
            rows.append(row)
        
        sum_arr = []
        user_arr = []

rows = np.array(rows) 
newData_cols = ['TrackID', 'Predictor']
newData = pd.DataFrame(rows)
newData.columns = newData_cols
print(newData.head())
newData.to_csv('./not_matrix_fact_predictions_1211.csv', index=False)

   AlbumRating_MF  ArtistRating_MF  Genre1Rating_MF  Genre2Rating_MF  \
0       11.690904        12.474678        13.945157        12.992632   
1       77.389346        80.694513        83.869803        79.879048   
2       14.254794        13.852148        10.914258        11.422376   
3        4.574390         4.590136         4.152230         4.137116   
4       72.024521        69.910350        54.788498        57.453849   

   Genre3Rating_MF        TrackID    UserID    ItemID   AlbumID  ArtistID  \
0        13.243756  199810_208019  199810.0  208019.0  209288.0       0.0   
1        80.586610   199810_74139  199810.0   74139.0  277282.0  271146.0   
2        11.039877    199810_9903  199810.0    9903.0       0.0       0.0   
3         4.087868  199810_242681  199810.0  242681.0  190640.0  244574.0   
4        55.480790   199810_18515  199810.0   18515.0  146344.0   33168.0   

   ...  Genre2Rating  Genre3Rating  MinScore  MaxScore  MeanScore  VarScore  \
0  ...           0.0     